In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Solar Panel Project/data/"

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [4]:
train_generator = train_datagen.flow_from_directory(
    data_path+"train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    data_path+"valid",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    data_path+"test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 569 images belonging to 6 classes.
Found 140 images belonging to 6 classes.
Found 110 images belonging to 6 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [6]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    min_delta=1e-2,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 245s 10s/step - accuracy: 0.2475 - loss: 1.7539 - val_accuracy: 0.3359 - val_loss: 1.6037
Epoch 2/20
 1/17 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.4375 - loss: 1.4390

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 699ms/step - accuracy: 0.4375 - loss: 1.4390 - val_accuracy: 0.2500 - val_loss: 1.6492
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 12s 237ms/step - accuracy: 0.4178 - loss: 1.4701 - val_accuracy: 0.4453 - val_loss: 1.4171
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5200 - loss: 1.3013 - val_accuracy: 0.6667 - val_loss: 0.9881
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 292ms/step - accuracy: 0.5032 - loss: 1.2363 - val_accuracy: 0.4922 - val_loss: 1.2998
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6250 - loss: 1.1554 - val_accuracy: 0.3333 - val_loss: 1.2845
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 234ms/step - accuracy: 0.5335 - loss: 1.1710 - val_accuracy: 0.4688 - val_loss: 1.2048
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 4.


In [10]:
print("Validation Performance:")
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

Validation Performance:
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.4803 - loss: 1.3554
Validation Loss: 1.3840829133987427
Validation Accuracy: 0.4642857015132904


In [11]:
print("Test Performance:")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Performance:
4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 9s/step - accuracy: 0.5088 - loss: 1.4011
Test Loss: 1.3324042558670044
Test Accuracy: 0.5454545617103577


In [12]:
Y_pred = model.predict(test_generator)
y_pred = Y_pred.argmax(axis=1)

print("Confusion Matrix:")
print(confusion_matrix(test_generator.classes, y_pred))

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 260ms/step
Confusion Matrix:
[[ 6  2  4  0  0  3]
 [ 3 15 15  2  0  3]
 [ 1  0 25  0  0  4]
 [ 0  0  1  5  0  0]
 [ 8  0  0  2  0  0]
 [ 0  0  2  0  0  9]]


In [13]:
print("\nClassification Report:")
print(classification_report(test_generator.classes, y_pred, target_names=train_generator.class_indices.keys()))


Classification Report:
                   precision    recall  f1-score   support

        Bird-drop       0.33      0.40      0.36        15
            Clean       0.88      0.39      0.55        38
            Dusty       0.53      0.83      0.65        30
Electrical-damage       0.56      0.83      0.67         6
  Physical-Damage       0.00      0.00      0.00        10
     Snow-Covered       0.47      0.82      0.60        11

         accuracy                           0.55       110
        macro avg       0.46      0.55      0.47       110
     weighted avg       0.57      0.55      0.51       110



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
